# Baby Names and Home Run Leaders

In [13]:
# dependencies 
import pandas as pd 
import splinter
from sqlalchemy import create_engine
import pymysql
pymysql.install_as_MySQLdb()

### Census Data for Baby Names Extraction and Transformation

In [14]:
# read csv and convert to df
names = pd.read_csv("usa_names_agg.csv")
names_df = pd.DataFrame(names)
# initial cleaning of df for just 2012 and 2013
clean_2012_df = names_df.loc[names_df['year'] == 2012]
clean_2013_df = names_df.loc[names_df['year'] == 2013]
names_df = pd.read_csv("usa_names_agg.csv")
# merging 2012 and 2013 df 
merged_names_df = pd.merge(clean_2012_df, clean_2013_df, on="name")
# cleaning up new df columns and names with the head displayed at the end to confirm
clean_merge_df = merged_names_df.drop(columns=["Unnamed: 0_x", "Unnamed: 0_y", "year_x", "year_y"])
new_df = clean_merge_df[['name', 'number_x', 'number_y']]
new_df.rename(columns={'number_x': '2012_occurances', 'number_y': '2013_occurances'}, inplace=True)
grouped_names_df = new_df.groupby(["name"]).sum()
baby_names_df = grouped_names_df.reset_index()
baby_names_df.head()

,name,2012_occurances,2013_occurances
0,Aaden,167,154
1,Aadhya,66,135
2,Aadi,22,29
3,Aadya,58,69
4,Aahana,22,15


### Home Run Leaders for 2012 Names Extraction and Transformation

In [15]:
# URL and html reading to gather hr hitter information
url = 'http://www.espn.com/mlb/stats/batting/_/year/2012/sort/homeRuns/order/true'
table = pd.read_html(url)
# creating initial df 
df = table[0]
df.dropna()
df.columns=['RK', 'PLAYER', 'TEAM','AB','R','H','2B','3B','HR','RBI','SB','CS','BB','SO','AVG','OBP','SLG','OPS','WAR']
# cleaning df 
# new data frame with split value columns 
cleaner = df["PLAYER"].str.split(" ", n = 1, expand = True)  
# making seperate first name column from new data frame 
df["name"]= cleaner[0]   
# making seperate last name column from new data frame 
df["last_name"]= cleaner[1]   
# Dropping old Name columns 
df.drop(columns =(['PLAYER','RK','TEAM','AB','R','H','2B',\
                   '3B','RBI','SB','CS','BB','SO','AVG','OBP',\
                   'SLG','OPS','WAR']), inplace = True) 
df = df.iloc[2:]
df = df[["name", "last_name", "HR"]]
clean_df = df.reset_index()
cleaner_df = clean_df.drop(columns=(['index']))
homerun_df = cleaner_df[cleaner_df.name != 'PLAYER']
homerun_df.head()

,name,last_name,HR
0,Miguel,Cabrera,44
1,Josh,Hamilton,43
2,Curtis,Granderson,43
3,Edwin,Encarnacion,42
4,Adam,Dunn,41


### Local Database Connection 

In [16]:
rds_connection_string = "root:R@cs1929@127.0.0.1/names_db"
engine = create_engine(f'mysql://{rds_connection_string}')

### Load DF into Local Database

In [18]:
# loading baby names df 
baby_names_df.to_sql(name='baby_names', con=engine, if_exists='append', index=False)
# loading hr hitter df 
homerun_df.to_sql(name='homerun_names', con=engine, if_exists='append', index=False)

### Confirm Data has been Loaded

In [19]:
# Baby names table
pd.read_sql_query('select * from baby_names', con=engine).head()

,name,2012_occurances,2013_occurances
0,Aaden,167,154
1,Aadhya,66,135
2,Aadi,22,29
3,Aadya,58,69
4,Aahana,22,15


In [20]:
# HR Hitter table 
pd.read_sql_query('select * from homerun_names', con=engine).head()

,name,last_name,HR
0,Miguel,Cabrera,44
1,Josh,Hamilton,43
2,Curtis,Granderson,43
3,Edwin,Encarnacion,42
4,Adam,Dunn,41
